In [ ]:
# things to do/doubts/discussions
# model sometimes give constant loss over epochs and gives poor results on prediction
# is the loss value supposed to reduce continously
# should we do augmentation of validation data ?
# All values are already between -1 to +1. should we do scaling on top of this ?
# if we are adding scaling of data, how to ensure that when it is being tested would give out rescaled data


# how to save subclassing api
# hyperparameter tuning
# how to save the best model among all the epochs
# should we shuffle repeat, prefetch etc ?
# incase of early stopping - what metric is to be monitored - val loss or val mse ?
from utility import *

In [ ]:
#load the data
data_size = 1000
X_train, Y_train, X_validation, Y_validation, X_test = load_data(data_size)

# defining the parameters

# loss
# Mean Squared Error
# Root Mean Squared Error
# Mean Absolute Error
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
batch_size = 32
epochs = 100
# steps_per_epoch = sum(train_occurences) / batch_size
starter_learning_rate = 1e-1
end_learning_rate = 1e-8
decay_steps = epochs * 3
# loss = tf.keras.losses.MeanAbsoluteError()
# loss = tf.keras.losses.MeanSquaredError()
loss = 'mean_squared_error'
metrics = tf.keras.metrics.MeanSquaredError()
scheduler = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate= starter_learning_rate,
    decay_steps= decay_steps,
    end_learning_rate= end_learning_rate,
    power=1)
# scheduler = 0.01
# optimizer=tf.keras.optimizers.Adam(learning_rate=scheduler)
kernel_regularizer=tf.keras.regularizers.L1L2(0.01)
callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error',patience=100)]
initializer=tf.keras.initializers.HeUniform()

verbose=2

# generating augmented data
X_train_Aug, Y_train_Aug = data_augmentation(X_train, Y_train)

In [ ]:
# NN Model
class NNmodel():
    def model():
        model = tf.keras.models.Sequential()
        model.add(Hidden_layer(4, kernel_regularizer=kernel_regularizer, initializer=initializer))
        model.add(Output_layer(2, kernel_regularizer=kernel_regularizer, initializer=initializer))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=scheduler),
                    loss = tf.keras.losses.MeanAbsoluteError(),
                    metrics = tf.keras.metrics.MeanSquaredError()
        )
        return model

In [ ]:
# Model 1 - NN fitting with Normal data
model = NNmodel.model()
history = model.fit(X_train, Y_train, 
                    epochs=epochs, 
                    batch_size= batch_size, 
                    verbose=verbose, 
                    validation_data=(X_validation, Y_validation),
                    callbacks=callbacks,
                    # shuffle=True
                    )  
model_eval(model, history, X_validation, Y_validation, X_train, Y_train, batch_size, X_test)
model.save('NN_1000_normal.h5')

In [ ]:
# Model 2 - NN fitting with Augmented data

model = NNmodel.model()
history = model.fit(X_train_Aug, Y_train_Aug,
                    epochs=epochs, 
                    batch_size= batch_size, 
                    verbose=verbose, 
                    validation_data=(X_validation, Y_validation),
                    callbacks=callbacks,
                    # shuffle=True
                    )  
model_eval(model, history, X_validation, Y_validation, X_train_Aug, Y_train_Aug, batch_size, X_test)
model.save('NN_1000_Aug.h5')

In [ ]:
# Equivariant NN Model
class EqNNmodel():
    def model():
        model = tf.keras.models.Sequential()
        model.add(EqHidden_layer(4, kernel_regularizer=kernel_regularizer, initializer=initializer))
        model.add(EqOutput_layer(2, kernel_regularizer=kernel_regularizer, initializer=initializer))

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=scheduler),
                    loss = tf.keras.losses.MeanAbsoluteError(),
                    metrics = tf.keras.metrics.MeanSquaredError()
        )
        return model

In [ ]:
# Model 3 Equivariant NN - on Normal data
Eqmodel = EqNNmodel.model()
history = Eqmodel.fit(X_train, Y_train,
                    epochs=epochs, 
                    batch_size= batch_size, 
                    verbose=verbose, 
                    validation_data=(X_validation, Y_validation),
                    callbacks=callbacks,
                    # shuffle=True
                    )  
model_eval(Eqmodel, history, X_validation, Y_validation, X_train, Y_train, batch_size, X_test)
Eqmodel.save('ENN_1000_normal.h5',
          overwrite=True)

In [ ]:
# Model 4 Equivariant NN - on Augmented data
model = EqNNmodel.model()
history = model.fit(X_train_Aug, Y_train_Aug,
                    epochs=epochs, 
                    batch_size= batch_size, 
                    verbose=verbose, 
                    validation_data=(X_validation, Y_validation),
                    callbacks=callbacks,
                    # shuffle=True
                    )  
model_eval(model, history, X_validation, Y_validation, X_train_Aug, Y_train_Aug, batch_size, X_test)
model.save('ENN_1000_Aug.h5')

In [ ]:
# models = ['NN_1000_normal.h5','NN_1000_Aug.h5','ENN_1000_normal.h5','ENN_1000_Aug.h5', ]
# for model in models:
import tensorflow_hub as hub
from keras.utils import CustomObjectScope

with CustomObjectScope({'Hidden_layer':hub.KerasLayer}):
       load_model = tf.keras.models.load_model('NN_1000_normal.h5')